# Welcome !

This may be the first time using Jupyter notebooks, a great tool for reproducible research. In addition to providing a a web application for coding in Python, Julia and R (and beyond), they allow one to directly embed:

 - text (as you may have realized already);
 - figures;
 <img src="MiCM_logo.png" width="400">
 
 - URL links towards e.g. the [Github repository](https://github.com/DylanMannKrzisnik/MiCM_W2021_Jupyter.git) housing the scripts and data needed for this workshop;
 - and so much more, like Youtube videos, PDF documents and - more relevant to this workshop - graphs and plots for visualizing your data and results
 

## Jupyter, an interactive Python command shell (i.e. IPython)

The IPython shell offers users an interactive environment, enabling easy navigation through *directories* and access to data. We can use many of the **BASH** commands which we'd usually run in a Terminal:

<img src="BASH_terminal.png" width="800">

So, rather than listing the content of our directory by executing the *'ls'* BASH command in a Terminal, we can execute *'ls'* directly from a *code cell*:

In [2]:
ls

BASH_terminal.png              README.md~
LICENSE                        micm_notes.txt
MiCM_2021_notebook.ipynb       requirements.txt
MiCM_2021_notebook_full.ipynb  sub-010321_EC_downsamp.fdt
MiCM_logo.png                  sub-010321_EC_downsamp.set
README.md


In [8]:
cat README.md

# MiCM 2021 - Intro to Jupyter & BASH
Introductory MiCM workshop on Jupyter notebooks with Binder image. Workshop held on April 23, 2021.

Access Binder/Docker image by clicking on badge below:

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/DylanMannKrzisnik/MiCM_W2021_Jupyter.git/main?filepath=MiCM_2021_notebook.ipynb)


In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = mne.io.read_raw_eeglab("sub-010321_EC.set")

In [ ]:
%load_ext watermark

In [ ]:
data.plot()

In [ ]:
from sklearn.decomposition import PCA
from mne.decoding import UnsupervisedSpatialFilter

In [ ]:
data_ts = data.get_data().T[:,:,np.newaxis]

In [ ]:
print("Original shape: ", data.get_data().shape)
print("New shape: ", data_ts.shape)

In [ ]:
pca = UnsupervisedSpatialFilter(PCA(30), average=False)
pca_data_ts = pca.fit_transform(data_ts).squeeze()

In [ ]:
pca_data_ts.shape

In [ ]:
plt.figure()
plt.plot(pca_data_ts)
plt.show()

In [ ]:
from mne.preprocessing import ICA

In [ ]:
%load_ext watermark

In [ ]:
%watermark --iversions